In [2]:
from numba import jit, cuda 
from scipy.signal import convolve2d
from scipy.integrate import ode
import numpy as np
from PIL import Image as img

In [7]:
# hàm trạng thái
def hamTrangThai(t, x, u, quanThe, n, m):
    A1 = taoMaTranDoiXungPerceptron(quanThe[:5])
    A2 = []
    for j in range(1,10):
        A2.append(taoMaTranDoiXungPerceptron(quanThe[5*j:5*j+5]))
    B1 = taoMaTranDoiXungPerceptron(quanThe[50:55])
    B2 = []
    for j in range(1,10):
        B2.append(taoMaTranDoiXungPerceptron(quanThe[5*j+50:5*j+55]))
    I = quanThe[100]
    x = x.reshape(n, m)
    dx = convolve2d(u, B1, 'same') + convolve2d(hamDauRa(x), A1, 'same') + tichChapBacHai(u, B2) + tichChapBacHai(hamDauRa(x), A2) + I
    return dx.reshape(n*m)

# Hàm tính tích chập bậc hai chạy trên GPU thông qua thư viện Numba (vì chạy trên CPU quá chậm)
@jit(target_backend='cuda')
#def tichChapBacHai_numba(y, A):
def tichChapBacHai(y, A):
    m, n = y.shape
    ketQua = np.zeros(y.shape) # tạo ma trận 0 có kích thước bằng ma trận y để lưu kết quả
    for i in range(m):
        for j in range(n):
            # xét từng điểm trên ma trận y kích thước m, n
            for k in range(3):#0.. 2
                for l in range(3):#0.. 2
                    # xét từng ma trận trong bộ A21 đến A29 (trong code là A[0] đến A[8])
                    x = k*3+l # ma trận A[x] đang được xét
                    for r in range(-1,2):#-1.. 1
                        for s in range(-1,2):#-1.. 1
                            # nhân từng số của ma trận láng giềng y(i,j;r,s) với ma trận A[x]
                            if(0<=i+r<m and 0<=j+s<n and 0<=i+1-k<m and 0<=j+1-l<n):
                                ketQua[i][j]+=A[x][r][s]*y[i+1-k][j+1-l]*y[i+r][j+s]
    return ketQua

# hàm đầu ra
def hamDauRa(x):
    return 0.5 * (np.abs(x + 1) - np.abs(x - 1))

# Tạo ma trận đối xứng từ 5 trọng số (hàm này chỉ dành riêng cho ma trận đối xứng 3x3)
def taoMaTranDoiXung(arr1):#giả sử cho mảng [1,2,3,4,5]
    arr2 = arr1[-2::-1].copy()#tạo mảng phụ [4,3,2,1]
    #arr2[0], arr2[2] = arr2[2], arr2[0]#nếu có thì sẽ đối xứng theo đường chéo phụ, nếu không thì sẽ là đối xứng tâm
    #print(f'arr1 = {arr1}, arr2 = {arr2}')
    return np.append(arr1,arr2).reshape(h,h)#kết hợp hai mảng rồi chuyển về dạng ma trận 3x3

# Trong công thức biến đổi từ mạng Nơ-ron phẩn hồi sang truyền thẳng, trọng số và đầu vào , đầu ra phải được nhân với nhau theo thứ tự nhất định (xem lại công thứ)
# hàm sau đây sẽ sắp xếp lại các số trọng bộ ma trận trọng số để tương ứng với công thức.
def taoMaTranDoiXungPerceptron(arr1):
    return taoMaTranDoiXung(np.array([arr1[2],arr1[1],arr1[3],arr1[0],arr1[4]]))

# Hàm tạo số ngẫu nhiên từ -9.99 đến 9.99
def ngauNhien():
    return np.random.randint(-999,1000)/100.

# Hàm tính sai số
def saiSo(d,y):
    return np.sum(0.5*(d-y)**2)

# Bán kính lân cận của nơ ron tế bào
r = 1

# kích thước của ma trận điều khiển và ma trận phản hồi (hai ma trận này luôn vuông)
h = 2*r + 1 # h = 3

class CNN:

    slts = 101 # số lượng trọng số trong quần thể

    # Quần thể trọng số (gồm 11 số)
    quanThe = np.array([ngauNhien() for i in range(slts)])
    # quanThe = np.array([0.,0.,0.,0.,2.]*10+[0.,0.,0.,0.,0.]*10+[-1.])

    # ma trận ảnh đầu vào u
    u = np.array(img.open('images/u1.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0

    # kích thước ma trận đầu vào, ma trận đầu ra và ma trận trạng thái
    m, n = u.shape
    print(m,n)

    # ma trận đầu ra mong muốn d
    d = np.array(img.open('images/d1.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0

    # hai giá trị ban đầu cho ptvp gồm x0 = [ma trận toàn số 0] với t0 = 0.0 và t1 để tính đầu ra x1 (t1 != t0)
    x0, t0, t1 = u*0., 0., 0.5

    # ma trận đầu ra y (y được tính theo x)
    y = hamDauRa(x0)

    # sai số cho phép
    delta_W0 = 0.000001

    # Tốc độ học của thuật toán
    alpha = 0.01

cnn = CNN()

# In ra bộ ma trận trọng số:
def print_A_B_I():
    print(f'A = {taoMaTranDoiXung(cnn.quanThe[:5])}')
    print(f'B = {taoMaTranDoiXung(cnn.quanThe[5:10])}')
    print(f'I = {cnn.quanThe[10]}')

# tạo thêm biên gồm toàn số 0 bao quanh ma trận a để tránh tràn biên
def them_bien(a):
    b = np.zeros(shape=[a.shape[0]+2,a.shape[1]+2])
    for i in range(1,a.shape[0]+1):
        for j in range(1,a.shape[1]+1):
            b[i][j] = a[i-1][j-1]
    return b

# Hàm để lấy trọng số từ file trongSoBac2ToiUu rồi lưu vào cnn.quanThe
def getQuanThe(i):# lấy bộ trọng số thứ i
    A1 = np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/A1.txt')
    A2 = []
    for j in range(1,10):
        A2.append(np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/A2{j}.txt'))
    B1 = np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/B1.txt')
    B2 = []
    for j in range(1,10):
        B2.append(np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/B2{j}.txt'))
    I = np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/I.txt')
    qt = [A1[0][0],A1[0][1],A1[0][2],A1[1][0],A1[1][1]]
    for A2x in A2:
        qt += [A2x[0][0],A2x[0][1],A2x[0][2],A2x[1][0],A2x[1][1]]
    qt += [B1[0][0],B1[0][1],B1[0][2],B1[1][0],B1[1][1]]
    for B2x in B2:
        qt += [B2x[0][0],B2x[0][1],B2x[0][2],B2x[1][0],B2x[1][1]]
    qt += [I]
    cnn.quanThe = np.array(qt)

# tính x, tính y, tính wij và cập trọng số
def capNhatTrongSo():
    ptvp.set_initial_value(cnn.x0.flatten(), 0.)
    ptvp.set_f_params(cnn.u, cnn.quanThe, cnn.n, cnn.m)
    assert ptvp.successful()
    x = (ptvp.integrate(cnn.t1))[:].reshape(cnn.n, cnn.m)
    cnn.y = hamDauRa(x)
    cnn.y = np.sign(cnn.y)
    y = them_bien(cnn.y) # thêm biên 0 cho y để tránh tràn biên
    u = them_bien(cnn.u) # thêm biên 0 cho u để tránh tràn biên
    delta_W = 0.
    for i in range(1,cnn.m+1):
        for j in range(1,cnn.n+1):
            Ys = [
                y[i-1][j] + y[i+1][j], 
                y[i][j-1] + y[i][j+1],
                y[i-1][j-1] + y[i+1][j+1],
                y[i-1][j+1] + y[i+1][j-1],
                y[i][j]
            ]
            for k in range(0,9):
                Ys += [
                    (y[i-1][j] + y[i+1][j]) * y[k//3][k%3],
                    (y[i][j-1] + y[i][j+1]) * y[k//3][k%3],
                    (y[i-1][j-1] + y[i+1][j+1]) * y[k//3][k%3],
                    (y[i-1][j+1] + y[i+1][j-1]) * y[k//3][k%3],
                    y[i][j] * y[k//3][k%3]
                ]
            Ys += [
                u[i-1][j] + u[i+1][j], 
                u[i][j-1] + u[i][j+1],
                u[i-1][j-1] + u[i+1][j+1],
                u[i-1][j+1] + u[i+1][j-1],
                u[i][j]
            ]
            for k in range(0,9):
                Ys += [
                    (u[i-1][j] + u[i+1][j]) * u[k//3][k%3],
                    (u[i][j-1] + u[i][j+1]) * u[k//3][k%3],
                    (u[i-1][j-1] + u[i+1][j+1]) * u[k//3][k%3],
                    (u[i-1][j+1] + u[i+1][j-1]) * u[k//3][k%3],
                    u[i][j] * u[k//3][k%3]
                ]
            Ys += [1]
            Ys = np.array(Ys)
            norm_n = np.sum(Ys**2)
            wij = cnn.alpha * (cnn.d[i-1][j-1] - y[i][j]) / norm_n
            delta_W += wij
            if(wij != 0.):
                for ts in range(cnn.slts):
                    cnn.quanThe[ts] += wij * Ys[ts]
    return delta_W

ptvp = ode(hamTrangThai) 
ptvp.set_integrator('vode') # thuật toán được sử dụng để giải ptvp, đọc thêm trong tài liệu
ptvp.set_initial_value(cnn.x0.flatten(), 0.) # lệnh flatten dùng để biến đổi ma trận về dạng mảng (đầu vào của ode phải là một số hoặc mảng)

getQuanThe(2)
print(f'slts: {cnn.slts}')
print_A_B_I()
print(f'u = {cnn.u}')
print(f'd = {cnn.d}')

ptvp.set_f_params(cnn.u, cnn.quanThe, cnn.n, cnn.m)
assert ptvp.successful()
x = (ptvp.integrate(cnn.t1))[:].reshape(cnn.n, cnn.m)
y = hamDauRa(x)
cnn.y = np.sign(y)
print(f'y = {cnn.y}')
print(f'Sai số: E = {saiSo(cnn.d,cnn.y)}')

8 8
slts: 101
A = [[-5.59  0.73  0.  ]
 [ 0.    7.07  0.  ]
 [ 0.    0.73 -5.59]]
B = [[ 0.   -2.6  -9.09]
 [-5.58  3.   -5.58]
 [-9.09 -2.6   0.  ]]
I = 0.0
u = [[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]
d = [[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1.  1.]
 [ 1.  1. -1.  1.  1.  1. -1.  1.]
 [ 1.  1. -1.  1. -1. -1. -1.  1.]
 [ 1.  1. -1.  1. -1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]


d:\anaconda\anaconda3\lib\site-packages\numba\core\ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'A' of function 'tichChapBacHai'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "C:\Users\phamd\AppData\Local\Temp\ipykernel_22752\2885260570.py", line 18:
@jit(target_backend='cuda')
#def tichChapBacHai_numba(y, A):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


y = [[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1. -1.  1. -1.  1. -1.  1.  1.]
 [ 1.  1.  1. -1.  1.  1. -1.  1.]
 [ 1.  1. -1.  1.  1.  1. -1.  1.]
 [ 1.  1. -1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1.  1. -1.  1.  1.  1.]
 [ 1.  1.  1. -1.  1.  1.  1.  1.]
 [ 1.  1. -1.  1.  1. -1.  1.  1.]]
Sai số: E = 22.0


In [8]:
print('Bộ trọng số ban đầu: ')
print_A_B_I()
# vòng lặp lớn
loop = 10000 #số lần chạy thử tối đa
count = 0 # đếm số lần chạy
while(True):
    # kiểm tra ptvp ODE
    if(not ptvp.successful()):
        print('Lỗi phương trình vi phân')
        break

    # đếm số lần chạy
    if(count>loop):
        print('Đạt đến số vòng lặp tối đa')
        break
    else:
        print(f'Lần chạy thứ {count+1}')
    delta_W = capNhatTrongSo()

    # kiểm tra sai số
    if(delta_W <= cnn.delta_W0):
        print('Đạt đến mức sai số cho phép')
        break

    count+=1

    print_A_B_I()
    print(f'Sai số hiện tại: w = {delta_W}')
    print(f'Sai số: E = {saiSo(cnn.d,cnn.y)}')

print('Bộ trọng sau khi tối ưu: ')
print_A_B_I()
print(f'Sai số: E = {saiSo(cnn.d,cnn.y)}')

Bộ trọng số ban đầu: 
A = [[-5.59  0.73  0.  ]
 [ 0.    7.07  0.  ]
 [ 0.    0.73 -5.59]]
B = [[ 0.   -2.6  -9.09]
 [-5.58  3.   -5.58]
 [-9.09 -2.6   0.  ]]
I = 0.0
Lần chạy thứ 1
Đạt đến mức sai số cho phép
Bộ trọng sau khi tối ưu: 
A = [[-5.58914647e+00  7.32131676e-01  1.95347541e-04]
 [-1.42754663e-03  7.06661106e+00 -1.42754663e-03]
 [ 1.95347541e-04  7.32131676e-01 -5.58914647e+00]]
B = [[ 0.   -2.6  -9.09]
 [-5.58  3.   -5.58]
 [-9.09 -2.6   0.  ]]
I = 0.0
Sai số: E = 22.0
